# Additional Information
> This Notebook contains information on use of _deepflash2_.

## Required Data Structure and Naming

### Ground Truth Estimation 
- __One parent folder__
- __One folder per expert__
- __Identical names for segmentations__

_Examplary structure:_

* [folder] parent_folder
    * [folder] expert1
        * [file] mask1.png
        * [file] mask2.png
    * [folder] expert1
        * [file] mask1.png
        * [file] mask2.png

### Training

#### Data structure
- __One folder for training images__
- __One folder for segmentation masks__

_Examplary structure:_
* [folder] images
  * [file] 0001.tif
  * [file] 0002.tif
* [folder] masks
  * [file] 0001_mask.png
  * [file] 0002_mask.png
  
#### Naming
- __Images must have unique name or ID__
    - _0001.tif --> name/ID: 0001; img_5.png --> name/ID: img_5, ..._ 
- __Corresponding masks must start with name or ID + a mask suffix__
    - _0001 -> 0001_mask.png (mask_suffix = "_mask.png")_
    - _0001 -> 0001.png (mask_suffix = ".png")_
    - mask suffix is inferred automatically 

### Prediction

- __One folder for training images__
- __One folder containing trained models (ensemble)__

_Examplary structure:_
* [folder] images
  * [file] 0001.tif
  * [file] 0002.tif
* [folder] ensemble
  * [file] unext50_deepflash2_model-1.pth
  * [file] unext50_deepflash2_model-2.pth
  
#### Naming
- __Images must have unique name or ID__
    - _0001.tif --> name/ID: 0001; img_5.png --> name/ID: img_5, ..._ 
- __Ensemble folder and models will be created during Training__
    - Do not change the naming of the models
    - If you want to train different ensembles, simply rename the ensemble folder

## Train-validation-split

The train-validation-split is defined as _[k-fold cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)_ with `n_splits`
- `n_splits` is the minimum of: (number of files in dataset,  `max_splits` (default:5))
- By default, the number of models per ensemble is limited to `n_splits`

_Example for a dataset containing 15 images_
- `model_1` is trained on 12 images (3 validation images) 
- `model_2` is trained on 12 images (3 different validation images) 
- ...
- `model_5` is trained on 12 images (3 different validation images) 

_Example for a dataset containing 2 images_
- `model_1` is trained on 1 image (1 validation image) 
- `model_2` is trained on 1 images (1 different validation image) 
- Only two models per ensemble